<img src='https://www.met.no/om-oss/logo/_/image/73f29cde-219f-487b-809c-9cdd61032c78:2efc46ce776f5f5337c4b0156ae0cbaa3b6bf6fe/width-768/Met_RGB_Horisontal.jpg' width=200 align=right>
<img src='https://raw.githubusercontent.com/norkyst/norkyst-logo/refs/heads/main/png/horizontal_35_91_100.png' width=200 align=right>

# __UNDER CONSTRUCTION__
# Stability ratio and Turner angle

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import xarray as xr

In [2]:
path = "https://thredds.met.no/thredds/dodsC/fou-hi/norkystv3_800m_m00_be"

ds = xr.open_dataset(path)

In [4]:
ds = ds.isel(X=slice(0,1500), Y=slice(500,1000))

In [5]:
def tad(salt, temp, z_rho, maxdepth=-100.0, resolution=100):

    """
    2025-04-23, kaihc@met.no
    
    Usage:
    
        turner_angle_distribution, zout = tad(salt, temp, z_rho)
        
    Input variables:

        salt                        - 4D salinity field from ROMS (ndarray [T,Z,Y,X])
        temp                        - 4D temperature field from ROMS (ndarray [T,Z,Y,X])
        z_rho                       - 4D depth values of rho-points (ndarray [T,Z,Y,X])
        maxdepth                    - maximum depth for return values
        resolution                  - number of fixed depths

    Output:

        turner_angle_distribution  - median and percentiles [5, 25, median, 75, 95] Turner angle values [deg]
        zout                       - corresponding ndarray with predetermined depth values [m]   
    """

    import gsw
    from scipy.interpolate import interp1d

    # Shift zeta to zero everywhere
    for i in range(z_rho.shape[1]):
        z_rho[:,i,:,:] -= z_rho[:,-1,:,:]

    # Reshape arrays to [Z, (T * Y * X)]
    Ncolumns = int(np.size(z_rho)/z_rho.shape[1])
    Nlevels = z_rho.shape[1]

    z_rho = np.transpose(z_rho, (1,0,2,3)).reshape((Nlevels, Ncolumns))
    salt = np.transpose(salt, (1,0,2,3)).reshape((Nlevels, Ncolumns))
    temp = np.transpose(temp, (1,0,2,3)).reshape((Nlevels, Ncolumns))

    # Remove columns on land and reverse layers so that surface comes first
    not_nan_ind = np.where(~np.isnan(z_rho[0,:]))[0]
    z_rho = z_rho[::-1, not_nan_ind]
    salt = salt[::-1, not_nan_ind]
    temp = temp[::-1, not_nan_ind]

    # Convert salinity and temperature from ROMS to TEOS-10 standards
    p = gsw.conversions.p_from_z(z_rho, 60.0)
    SA = gsw.conversions.SA_from_SP(salt, p, 0.0, 60.0)
    CT = gsw.conversions.CT_from_pt(SA, temp)

    # Get Turner angle
    Tu, Rs, pmid = gsw.Turner_Rsubrho(SA,CT,p)
    
    # Get z coordinates
    z = gsw.z_from_p(pmid, 60.0)

    # Interpolate to fixed depths
    zout = np.linspace(maxdepth,-1,resolution)
    Tu_interp = np.zeros((zout.shape[0], Tu.shape[1]))
    for i in range(Tu.shape[1]):
        f = interp1d(z[:,i], Tu[:,i], bounds_error=False, fill_value=np.nan)
        Tu_interp[:,i] = f(zout)

    # Loop over levels and compute percentiles
    Tu_out = np.zeros((zout.shape[0],5))

    for i in range(len(zout)):
        tu_vals = Tu_interp[i,:].ravel()
        Tu_out[i,:] = np.nanquantile(tu_vals, [0.05,0.25,0.50,0.75,0.95])


    return Tu_out, zout

In [7]:
maxdepth = -1000.0
resolution = 400
tu, z = tad(ds.salinity.values, ds.temp.values, ds.z_rho.values, maxdepth, resolution)

MemoryError: Unable to allocate 243. GiB for an array with shape (11591, 15, 500, 1500) and data type int16